In [1]:
from tensorflow.keras.models import load_model
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.callbacks import EarlyStopping

# Load the model
model = load_model("15_point_showcase_model.keras")



2024-07-30 12:46:47.752787: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-30 12:46:47.753458: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-30 12:46:47.794627: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-30 12:46:47.818328: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-30 12:46:47.847276: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been 

In [10]:
data = pd.read_csv("15_point_showcase.csv")

# Separate features and labels
X = data.drop(columns=["Material_Type"])
y = data["Material_Type"]

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [8]:
# SERIAL ############################

import numpy as np  # Add this line to import the NumPy library
import serial
from sklearn.preprocessing import LabelEncoder


# Open the serial port
ser = serial.Serial('COM3', baudrate=115200)  # Adjust the baudrate as needed

# Initialize a variable to store the received data
received_data = ""

# print(label_encoder.classes_)

try:
    while True:
        # Read a line of data from the serial port (assuming data is sent as lines)
        line = ser.readline().decode('utf-8').strip()

        # Split the received data into a list of numbers using a comma as the delimiter
        data_values = [float(val) for val in line.split(',')]

        # Normalize the data values if needed
        normalized_data = [(val - 0) / (8410000 - 0) for val in data_values]

        # print(data_values)

        # Make predictions on the normalized data
        predictions = model.predict([normalized_data])  # Convert normalized_data to a list

        # Format the model so that its easier to read in terminal 
        formatted_predictions = np.around(predictions, 4)

        # Get the top prediction
        top_prediction = np.argmax(predictions)

        top_plastic_type = label_encoder.inverse_transform([top_prediction])[0]
        # print(top_plastic_type)
        likelihood = predictions[0][top_prediction] * 100
        

        # if predictions.shape[1] == 6:
        #     plastic_types = ["HDPE", "LDPE", "PET", "PP", "PVC", "Unknown"]
        # else:
        #     plastic_types = ["PE", "PET", "PP", "PVC", "Unknown"]

        # plastic_types = ["PE", "PET", "PLA", "PP", "PVC", "Unknown"]
        
        # top_plastic_type = plastic_types[top_prediction]

        # likelihood = predictions[0][top_prediction] * 100


        # Display the prediction for manual input
        # print(f"Manual Input Prediction:")
        print(f"  Prediction: {top_plastic_type} with {likelihood:.2f}% likelihood")

        likelihood_str = round(likelihood)-1

        received_data = ""

        # Send the top plastic type back over serial
        # ser.write(top_plastic_type.encode('utf-8')) 
        # ser.write(str(likelihood_str).encode('utf-8'))
        data_to_send = f"{top_plastic_type},{likelihood_str}"  # Combine the data fields with a comma delimiter
        ser.write(data_to_send.encode('utf-8'))

except KeyboardInterrupt:
    # Handle Ctrl+C to stop the loop gracefully
    pass

finally:
    # Close the serial port when done
    ser.close()

# Now, you can use the 'received_data' variable for further processing
print("total data:\n", received_data)


1/1 [==============================] - 0s 98ms/step
  Prediction: Unknown with 100.00% likelihood
1/1 [==============================] - 0s 48ms/step
  Prediction: HDPE with 100.00% likelihood


SerialException: GetOverlappedResult failed (PermissionError(13, 'Access is denied.', None, 5))

In [11]:
# BLUETOOTH ############################

import numpy as np  # Add this line to import the NumPy library
import serial
from sklearn.preprocessing import LabelEncoder


# Open the serial port
ser = serial.Serial('COM5', baudrate=9600)  # Adjust the baudrate as needed

# Initialize a variable to store the received data
received_data = ""

# print(label_encoder.classes_)

try:
    while True:
        # Read a line of data from the serial port (assuming data is sent as lines)
        line = ser.readline().decode('utf-8').strip()

        # Split the received data into a list of numbers using a comma as the delimiter
        data_values = [float(val) for val in line.split(',')]

        print(line)
        
        # Normalize the data values if needed
        normalized_data = [(val - 0) / (8410000 - 0) for val in data_values]
        
        # Make predictions on the normalized data
        predictions = model.predict([normalized_data])  # Convert normalized_data to a list
        
        # Format the model so that its easier to read in terminal 
        formatted_predictions = np.around(predictions, 4)

        # Get the top prediction
        top_prediction = np.argmax(predictions)

        # top_plastic_type = label_encoder.inverse_transform([top_prediction])[0]
        # print(top_plastic_type)
        # likelihood = predictions[0][top_prediction] * 100
        

        if predictions.shape[1] == 6:
            plastic_types = ["HDPE", "LDPE", "PET", "PP", "PVC", "Unknown"]
        else:
            plastic_types = ["PE", "PET", "PP", "PVC", "Unknown"]
        
        top_plastic_type = plastic_types[top_prediction]

        likelihood = predictions[0][top_prediction] * 100


        # Display the prediction for manual input
        print(f"Manual Input Prediction:")
        print(f"  Prediction: {top_plastic_type} with {likelihood:.2f}% likelihood")

        received_data = ""

        # Send the top plastic type back over serial
        ser.write(top_plastic_type.encode('utf-8'))

except KeyboardInterrupt:
    # Handle Ctrl+C to stop the loop gracefully
    pass

finally:
    # Close the serial port when done
    ser.close()

# Now, you can use the 'received_data' variable for further processing
print("total data:\n", received_data)


In [ ]:
import numpy as np
import serial
import csv  # Import the csv module
from sklearn.preprocessing import LabelEncoder

# Open the serial port
ser = serial.Serial('COM3', baudrate=115200)  # Adjust the baudrate as needed

# Create and open a CSV file for writing
csv_file = open('adding.csv', 'w', newline='')
csv_writer = csv.writer(csv_file)

try:
    while True:
        # Read a line of data from the serial port (assuming data is sent as lines)
        line = ser.readline().decode('utf-8').strip()

        # Split the received data into a list of numbers using a comma as the delimiter
        data_values = [float(val) for val in line.split(',')]

        print(data_values)

        # Write the data values to the CSV file as a new row
        csv_writer.writerow(data_values)

except KeyboardInterrupt:
    # Handle Ctrl+C to stop the loop gracefully
    pass

finally:
    # Close the CSV file and the serial port when done
    csv_file.close()
    ser.close()

# The received data has been saved in the 'received_data.csv' file
print("Data saved to 'PP2.csv'")



[268903.0, 512993.0, 725651.0, 1189689.0, 1033021.0, 781697.0, 287735.0, 175749.0, 369745.0, 602367.0, 953183.0, 1117747.0, 918695.0, 535185.0, 230755.0]
[222561.0, 399473.0, 643977.0, 1487453.0, 1337153.0, 813703.0, 352855.0, 200005.0, 300139.0, 506009.0, 1062365.0, 1429275.0, 1105991.0, 605581.0, 268909.0]
[195783.0, 392477.0, 522813.0, 1283407.0, 1369731.0, 1085553.0, 351805.0, 231775.0, 279913.0, 448175.0, 934273.0, 1377553.0, 1268573.0, 729875.0, 295985.0]
[247193.0, 404169.0, 819885.0, 1909933.0, 918149.0, 595195.0, 313061.0, 250585.0, 321729.0, 597853.0, 1337781.0, 1405917.0, 751901.0, 496307.0, 265955.0]
[278821.0, 415827.0, 642585.0, 1562665.0, 1382683.0, 1275309.0, 395315.0, 159227.0, 329577.0, 516189.0, 1101479.0, 1478257.0, 1339605.0, 840233.0, 277217.0]
[206757.0, 351963.0, 749729.0, 992767.0, 1380817.0, 629337.0, 293611.0, 232597.0, 265397.0, 534377.0, 867367.0, 1236685.0, 1048935.0, 466125.0, 264985.0]
[1768827.0, 2914297.0, 4589969.0, 7242751.0, 6431417.0, 3714651.0, 18

SerialException: GetOverlappedResult failed (PermissionError(13, 'Access is denied.', None, 5))

In [ ]:
# B8D61A0C805E

# Incoming ################
# import serial

# # Replace 'COM4' with the COM port for your ESP32
# ser = serial.Serial('COM4', baudrate=9600)  # Adjust the baud rate as needed

# print('hello there')
# # received_data = ser.read()
# received_data = ser.readline().decode('utf-8').strip()
# print(received_data)
# ser.close()


# # Outgoing ###############
# import serial

# # Replace 'COM5' with the COM port for your ESP32
# ser = serial.Serial('COM4', baudrate=9600)  # Adjust the baud rate as needed

# data_to_send = "data is sending"
# ser.write(data_to_send.encode('utf-8'))
# ser.close()



KeyboardInterrupt: 

In [ ]:
import numpy as np  # Add this line to import the NumPy library
import serial
import csv  # Import the csv module
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import load_model
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping

data = pd.read_csv("likelihood PLA.csv", header=None)

# Separate features and labels
# X = data.iloc[:,1]
# y = data.iloc[:,0]

# Rename the columns for clarity
data.columns = ["labels", "accuracy"]

# Group the data by the "labels" column and calculate the mean accuracy for each group
grouped_data = data.groupby("labels")["accuracy"].count().reset_index()

# Print the grouped data
print(grouped_data)

# print(X)
# print(y_encoded)

  labels  accuracy
0    PLA        49
1    PVC         1
